# Numba

In [1]:
from numba import (
    __version__ as numba_version,
    jit,
    njit,
    TypingError,
)
from numpy import (
    __version__ as numpy_version,
    array,
)
from time import time

print("NumPy version of this slideshow is", numpy_version)
print("numba version of this slideshow is", numba_version)

NumPy version of this slideshow is 1.20.3
numba version of this slideshow is 0.54.1


# Getting started

[numba](https://numba.pydata.org/) is a library that provides JIT (just-in-time) and AOT (ahead-of-time) compiler for Python like [jax](https://github.com/google/jax).

It also allows you to write parallel code easily (like [OpenMP](https://www.openmp.org/) for C/C++/Fortran)
as well as [CUDA](https://developer.nvidia.com/cuda-toolkit).

It works well enough with [numpy](https://numpy.org/) but misses convenient [features](https://numba.pydata.org/numba-doc/dev/reference/numpysupported.html),
though they can be efficiently implemented with `numba`.

# Example

## Basic usage

You can use `@jit` decorator for your functions 

In [2]:
@jit
def f(x):
    return 0

The function is compiled just-in-time &mdash; only when you call it.
Now, the function doesn't have any assembly.

In [3]:
print(f.inspect_asm())

{}


Let us call it with `None` and see that now we have a signature for a tuple of arguments `(none, )`.

In [4]:
f(None)
print(list(f.inspect_asm().keys()))
# for v, k in f.inspect_asm().items():
#     print(v, k)

[(none,)]


Call it with integer parameter &mdash; you will have one more overload of the function.

In [5]:
f(0)
print(list(f.inspect_asm().keys()))

[(none,), (int64,)]


## Want to see more?

In [6]:
for v, k in f.inspect_asm().items():
    print(v, k)

(none,) 	.text
	.file	"<string>"
	.globl	_ZN8__main__5f$241Ev
	.p2align	4, 0x90
	.type	_ZN8__main__5f$241Ev,@function
_ZN8__main__5f$241Ev:
	movq	$0, (%rdi)
	xorl	%eax, %eax
	retq
.Lfunc_end0:
	.size	_ZN8__main__5f$241Ev, .Lfunc_end0-_ZN8__main__5f$241Ev

	.globl	_ZN7cpython8__main__5f$241Ev
	.p2align	4, 0x90
	.type	_ZN7cpython8__main__5f$241Ev,@function
_ZN7cpython8__main__5f$241Ev:
	.cfi_startproc
	subq	$24, %rsp
	.cfi_def_cfa_offset 32
	movq	%rsi, %rdi
	movabsq	$.const.f, %rsi
	movabsq	$PyArg_UnpackTuple, %r9
	leaq	16(%rsp), %r8
	movl	$1, %edx
	movl	$1, %ecx
	xorl	%eax, %eax
	callq	*%r9
	movq	$0, 8(%rsp)
	testl	%eax, %eax
	je	.LBB1_3
	movabsq	$_ZN08NumbaEnv8__main__5f$241Ev, %rax
	cmpq	$0, (%rax)
	je	.LBB1_2
	movabsq	$_ZN8__main__5f$241Ev, %rax
	leaq	8(%rsp), %rdi
	callq	*%rax
	movabsq	$PyLong_FromLongLong, %rax
	xorl	%edi, %edi
	callq	*%rax
	addq	$24, %rsp
	.cfi_def_cfa_offset 8
	retq
.LBB1_2:
	.cfi_def_cfa_offset 32
	movabsq	$PyExc_RuntimeError, %rdi
	movabsq	$".const.missing Envi

# Why should I care?

## Performance

You can write an ordinary though fast Python code.

Consider this strange implementation of Fibonacci numbers calculation.

In [7]:
def million_fibonacci(n):
    for j in range(1_000_000):
        if n < 1:
            return 0
        elif n == 1:
            return 1
        x, y = 0, 1
        for i in range(n - 1):
            x, y = y, x + y
    return y

It takes several seconds for the 30th number

In [8]:
start = time()
result = million_fibonacci(30)
print(f"The ordinary version running time is {time() - start:>.2}s")

The ordinary version running time is 3.1s


Let us decorate it by our powerful JIT!

Remember that
```python
@jit
def f():
    return
```
is the same as
```python
def f():
    return
f = jit(f)
```

In [9]:
million_fibonacci_jit = jit(million_fibonacci)
start = time()
assert result == million_fibonacci_jit(30)
print(f"First time JIT version running time is {time() - start:>.2}s")

First time JIT version running time is 0.17s


Maybe one more time?

In [10]:
start = time()
assert result == million_fibonacci_jit(30)
print(f"JIT version running time is {time() - start:>.2}s")

JIT version running time is 0.017s


Still don't believe?

In [11]:
start = time()
assert result == million_fibonacci_jit(30)
print(f"JIT version running time is {time() - start:>.2}s")

JIT version running time is 0.017s


## Potential type safety

We have already seen that `numba` can speed-up your calculations if the function is called multiple times.

Can we make it better? Sure, compile it when you create it by specifying types!

In [12]:
start = time()
@jit("uint64(uint64)")
def fibonacci(n):
    if n < 1:
        return 0
    elif n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)
print(f"The function is compiled in {time() - start:>.2}s")

The function is compiled in 0.14s


In [13]:
start = time()
result = fibonacci(30)
print(f"The first launch time is  {time() - start:>.2}s")
start = time()
assert result == fibonacci(30)
print(f"The second launch time is {time() - start:>.2}s")

The first launch time is  0.0077s
The second launch time is 0.0077s


We have seen the "type" part but where is the "safety"?

Here it is. Numba Will not allow you to call this function with arguments that cannot be safely converted to the type you have specified.

In [14]:
try:
    fibonacci(1.0)
except TypeError as e:
    print(e)

Ambiguous overloading for <function fibonacci at 0x7f60ec39e8b0> (float64,):
(int64,) -> int64
(uint64,) -> uint64


# Combining functions powered by JIT

## JIT and JIT again

All these type checks look good but don't they make the same bad things as ordinary type checks in CPython?

You can decrease the number of checks by applying JIT to the hierarchy of called functions.

In [15]:
@jit
def g(x):
    return million_fibonacci(x) + 1

In [16]:
start = time()
g(30)
print(f"The first launch time is  {time() - start:>.2}s")
start = time()
g(30)
print(f"The second launch time is  {time() - start:>.2}s")

/tmp/ipykernel_5699/2744060948.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "g" failed type inference due to: Untyped global name 'million_fibonacci': Cannot determine Numba type of <class 'function'>

File "../../../../tmp/ipykernel_5699/2744060948.py", line 3:
<source missing, REPL/exec in use?>

  @jit
/opt/conda/lib/python3.9/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "g" was compiled in object mode without forceobj=True.

File "../../../../tmp/ipykernel_5699/2744060948.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
/opt/conda/lib/python3.9/site-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#de

The first launch time is  3.9s
The second launch time is  5.0s


No, `numba` will not modify all the functions for you.

It's better to call functions decorated by `@jit` whithin such functions.

In [17]:
@jit
def g(x):
    return million_fibonacci_jit(x) + 1

In [18]:
start = time()
g(30)
print(f"The first launch time is  {time() - start:>.2}s")
start = time()
g(30)
print(f"The second launch time is  {time() - start:>.2}s")

The first launch time is  0.11s
The second launch time is  0.0084s


## Flags

Though it doesn't fail.

Numba even fails on wrong types but it cannot check whether it can optimize my code?

It can. Use `nopython=True`.

In [19]:
@jit(nopython=True)
def g(x):
    return million_fibonacci(x) + 1

In [20]:
try:
    g(30)
except TypingError as e:
    print(e)

Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'million_fibonacci': Cannot determine Numba type of <class 'function'>

File "../../../../tmp/ipykernel_5699/2017834750.py", line 3:
<source missing, REPL/exec in use?>



Want more?

You can use `njit` as an alias to `jit(nopython=True)`.

You can disable [GIL](https://wiki.python.org/moin/GlobalInterpreterLock) with `nogil=True`.
This will cause problems if you want to stop your function with `Ctrl+C` in Linux terminal.

You can even use `cache=True` to compile your function once and use it every time you launch your code again!
Just like AOT.

In [21]:
start = time()
@njit("uint64(uint64)", nogil=True)
def nfibonacci(n):
    if n < 1:
        return 0
    elif n == 1:
        return 1
    return nfibonacci(n - 1) + nfibonacci(n - 2)
print(f"The function is compiled in {time() - start:>.2}s")

The function is compiled in 0.23s


In [22]:
start = time()
fibonacci(30)
print(f"An ordinary `@jit` work time is     {time() - start:>.2}s")
start = time()
nfibonacci(30)
print(f"Work time with `@njit` and flags is {time() - start:>.2}s")

An ordinary `@jit` work time is     0.012s
Work time with `@njit` and flags is 0.013s


The difference is negligible in this example but `@njit` can watch what you use to avoid sad mistakes.

Let us look at some mistakes you can make with `nopython=False` (the default behavior of `@jit`).

In [23]:
@jit
def jit_sum(x):
    accumulator = 0
    for element in x:
        accumulator += element
    return accumulator

What if I use `list`? Remember that a list is not an array in general case; and lists in Python may contain data of different types.
If there was a way to perform all those checks on them more efficiently, CPython would definitely perform them.

In [24]:
print(jit_sum([1, 2, 3]))

6


/opt/conda/lib/python3.9/site-packages/numba/core/ir_utils.py:2139: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'x' of function 'jit_sum'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../tmp/ipykernel_5699/496693149.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


In [25]:
@jit
def jit_dict(x):
    for key, value in x.items():
        if key == "name":
            return value
    return ""

What's wrong with `dict`? The same thing as with `list`.

In [26]:
print(jit_dict({"name": "John"}))

/tmp/ipykernel_5699/3057356901.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "jit_dict" failed type inference due to: non-precise type pyobject
During: typing of argument at /tmp/ipykernel_5699/3057356901.py (3)

File "../../../../tmp/ipykernel_5699/3057356901.py", line 3:
<source missing, REPL/exec in use?>

  @jit


John


/opt/conda/lib/python3.9/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "jit_dict" was compiled in object mode without forceobj=True.

File "../../../../tmp/ipykernel_5699/3057356901.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
/opt/conda/lib/python3.9/site-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit

File "../../../../tmp/ipykernel_5699/3057356901.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaDeprecationWarning(msg,


What should we do?

Use `numpy`!

But how to avoid all those cases?Use `@njit`!

Use `@njit`!

Won't it check something only on call?

Set the function signature to save some time.

In [27]:
@njit("int64(int64[:])")
def njit_sum(x):
    accumulator = 0
    for element in x:
        accumulator += element
    return accumulator

In [28]:
njit_sum(array([1, 2, 3]))

6

What is `int64[:]`?

We will watch the next slide show about it.
It is a type for a one-dimensional array of `int64` entities.
You can see the same syntax in [Cython](https://cython.org/)'s
[Working with Python arrays](https://cython.readthedocs.io/en/latest/src/tutorial/array.html) and
[Typed Memoryviews](https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html).

# What's next?

- [Supported NumPy features](https://numba.pydata.org/numba-doc/dev/reference/numpysupported.html) to know how to use `numba` with your `numpy`-powered code.

- [Automatic module jitting with `jit_module`](https://numba.pydata.org/numba-doc/latest/user/jit-module.html) if you want not to decorate each function separately but to make the entire module work faster.

- [Compiling Python classes with `@jitclass`](https://numba.pydata.org/numba-doc/dev/user/jitclass.html) if you want to speed-up entire class.

- [Compiling code ahead of time (AOT)](https://numba.pydata.org/numba-doc/dev/user/pycc.html) if you don't want to wait for the compilation each time you launch your application and/or if you want to **hide** your code: this will produce a separate shared library that can be simply `import`-ed and used in your software like an ordinary Python module. Also, read **Limitations** section before using it. Read the documentation before using it. You've been warned.

**DISCLAIMER**: I'm **NOT SURE** how obfuscated the result of numba's AOT is, so don't think that it will be irreversibly compiled.